# Notebook 0a

In [1]:
import pandas as pd 
import itertools
import os
from pathlib import Path
import pickle

In [2]:
import Notebook_0_helper_functions as Nb_0

In [9]:
#parameters: 

# mouse = 'T240'
# data_day = '2019_09_27__T240'
# date = '2019_09_27'

# drop2blocks= 'yes'
# drop2lastblocks="yes"
# HowManyBack = 1

# data_dir_input = "/Users/gilmandelbaum/Desktop/T240_mouse"
# data_dir_output = "/Users/gilmandelbaum/Desktop/output/"


mouse = 'T196'
data_day = '2019_04_20__T196'
date = '2019_04_20'

drop2blocks= 'yes'
drop2lastblocks="yes"
HowManyBack = 1

data_dir_input = "/Users/gilmandelbaum/Desktop/mice_for_ts"
data_dir_output = "/Users/gilmandelbaum/Desktop/output/"


In [10]:
# stash result in a cache directory. for mouse: 
cache_dir = os.path.join(data_dir_output, mouse)
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
    
# stash result in a cache directory. for the session, i.e, the day. 
cache_dir = os.path.join(data_dir_output+'/'+mouse, data_day)
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)
    
# stash result in a cache directory, for how many back. 
cache_dir_plt = os.path.join(data_dir_output+'/'+mouse+'/'+data_day, str(HowManyBack)+'_Back')
if not os.path.exists(cache_dir_plt):
    os.makedirs(cache_dir_plt)

In [11]:
#1= reward. 2=non rewarded (HowManyBack*2 is the amount of previous trials that we will look at).
combinations =list(itertools.product([2,1], repeat=HowManyBack*2))

In [12]:
def combinations_string(combinations):
    total=[]
    for i in combinations:
        total.append(''.join(str(e) for e in i))    
    return total

In [13]:
combination_stringlist = combinations_string(combinations)

# Import behavior related data per trial

In [14]:
behavior_data_trial = pd.read_excel(data_dir_input+"/"+mouse+"/"+data_day+"/"+mouse+"_dataTrial_label.xlsx")

# Add columns to behavior data (per trial) 
This will help with the splitting into trial types.

In [15]:
behavior_data_trial["Date"]= date
behavior_data_trial["Mouse"]= mouse
behavior_data_trial=behavior_data_trial.set_index("nTrial")

In [16]:
behavior_data_trial = Nb_0.define_current_choice_direction(behavior_data_trial)

In [17]:
behavior_data_trial = Nb_0.get_previous_rewarded_or_not(behavior_data_trial, HowManyBack)

In [18]:
behavior_data_trial = Nb_0.get_previous_choice_direction_same_or_not(behavior_data_trial, HowManyBack)

In [19]:
#to drop first two blocks and last two blocks 
if drop2blocks=="yes":
    behavior_data_trial = behavior_data_trial[behavior_data_trial.iBlock > 2]
if drop2lastblocks=="yes":
    lastblock = behavior_data_trial['iBlock'].values.tolist()[-1]
    SecondToLast = lastblock-1
    behavior_data_trial = behavior_data_trial[behavior_data_trial.iBlock<SecondToLast]

In [20]:
root = Path(data_dir_output+"/"+mouse+"/"+data_day+'/'+str(HowManyBack)+"_Back")
d = mouse+"_"+date+"Notebook_0_a.pickle" 
my_path = root / d
my_file = open(my_path, 'wb')
my_file = pickle.dump(behavior_data_trial, my_file)